In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
import sys
from pathlib import Path
import xarray as xr
import cfgrib
import cartopy.crs as ccrs  # Projeções de mapas.
import cartopy.feature as cfeature  # Elementos geográficos.
from matplotlib.tri import Triangulation
import matplotlib.patches as patches
from typing_extensions import TypeAlias

#Import local libraries
import aux

In [2]:
year='2018'
month='12'
month_abbr = datetime.datetime.strptime(month, "%m").strftime("%b")

base_dir='/pesq/dados/monan/users/madeleine.gacita/global_data/'

In [9]:
rad_var_dict={
    "ISR": {
        "era5_name" : "ssrd",
        "era5_longname" :"surface_solar_radiation_downwards",
        "ceres_name" : "init_all_sfc_sw_dn", 
        "monan_name" : "swdnb",
        "unit" : "W m^{-2}",
        "label" : "Surface shortwave radiation downwards",
        "vmin" : 0,
        "24vmin" : 0,
        "vmax" : 1100,
        "24vmax" : 8000
    },
    "ISRC": {
        "era5_name" : "ssrdc",
        "era5_longname" :"surface_solar_radiation_downward_clear_sky",
        "ceres_name" : "init_clr_sfc_sw_dn", 
        "monan_name" : "swdnbc",
        "unit" : "W m^{-2}",
        "label" : "Surface shortwave radiation downwards (clear sky)",
        "vmin" : 0,
        "24vmin" : 0,
        "vmax" : 1300,
        "24vmax" : 8000
    },
    "OLR": {
        "era5_name": "ttr",
        "era5_longname" :"top_net_thermal_radiation",
        "ceres_name" : "init_all_toa_lw_up", 
        "monan_name" : "lwupt",
        "unit" : "W m^{-2}",
        "label" : "TOA Outgoing longwave radiation",
        "vmin" : -300,
        "vmax" : -150,
        "24vmin" : 0,
        "24vmax" : 0
    },
    "OLRC": {
        "era5_name" : "ttrc",
        "era5_longname" : "top_net_thermal_radiation_clear_sky",
        "ceres_name" : "init_clr_toa_lw_up", 
        "monan_name" : "lwuptc",
        "unit" : "W m^{-2}",
        "label" : "TOA Outgoing longwave radiation (clear sky)",
        "vmin" : -300,
        "vmax" : -150,
        "vmax" : 0,
        "24vmax" : 0
    },
    "TISR": {
        "era5_name" : "tisr",
        "era5_longname" : "toa_incident_solar_radiation",
        "ceres_name" : "toa_sw_insol", 
        "monan_name" : "swdnt",
        "unit" : "W m^{-2}",
        "label" : "TOA incident short-wave (solar) radiation",
        "vmin" : 0,
        "24vmin" : 0,
        "vmax" : 1300,
        "24vmax" : 8000
    }
}

surf_flux_dict={
    "HF": {
        "era5_name" : "sshf",
        "era5_longname" : "surface_sensible_heat_flux",
        "monan_name" : "hfx", 
        "unit" : "W/m**2",
        "label" : "Sensible heat flux"
    },
    "LF": {
        "era5_name": "sslf",
        "era5_longname":"surface_latent_heat_flux",
        "monan_name": "lf", 
        "unit": "W m^{-2}",
        "label": "Latent heat flux"
    }
}

profile_vars_dict={
    "ISR": {
        "monan_name" : "ssrd",
        "ceres_name" : "adj_all_sw_dn",
        "unit" : "W m^{-2}",
        "label" : "Incoming shortwave radiation"
    },
    "OSR": {
        "monan_name" : "ssrd",
        "ceres_name" : "adj_all_sw_up", 
        "unit" : "W m^{-2}",
        "label" : "Outgoing shortwave radiation"
    },
    "ILR": {
        "monan_name" : "ttrc",
        "ceres_name" : "adj_all_lw_dn", 
        "unit" : "W m^{-2}",
        "label" : "Incoming longwave radiation"
    },
    "OLR": {
        "monan_name" : "ttr",
        "ceres_name" : "adj_all_lw_up", 
        "unit" : "W m^{-2}",
        "label" : "Outgoing longwave radiation"
    }    
}

In [10]:
def apply_lon_lat_conventions(ds):
    # Renames
    if "lon" in ds.dims:
        ds = ds.rename({"lon": "longitude"})
    if "lat" in ds.dims:
        ds = ds.rename({"lat": "latitude"})
 
    # Flip latitudes (ensure they are monotonic increasing)
    if "latitude" in ds.dims:
        lats = ds["latitude"]
        if len(lats) > 1 and lats[0] > lats[-1]:
            ds = ds.reindex(latitude=ds.latitude[::-1])
 
    # Convert longitude to [-180, 180[
    if "longitude" in ds.dims and ds["longitude"].max() > 180:
        lons = ds["longitude"]
        lons_attrs = lons.attrs
        new_lons = np.concatenate([lons[lons >= 180], lons[lons < 180]])
        ds = ds.reindex(longitude=new_lons)
        ds = ds.assign_coords(longitude=(((ds["longitude"] + 180) % 360) - 180))
        ds["longitude"].attrs = lons_attrs
    return ds

# Opening CERES SYN_1deg Ed A

In [11]:
syn_file_arg = base_dir+"CER_SYN1deg-MHour/Terra-Aqua-MODIS_Edition4A/CER_SYN1deg-MHour_Terra-Aqua-MODIS_Edition4A_407406."+year+month+".hdf"
path = Path(syn_file_arg)
print(syn_file_arg)

if not path.exists():
    print(f"File does not exist: {path!s}")
else:
    from pyhdf.SD import SD, SDC
    sd = SD(str(path), SDC.READ)
    print("pyhdf.SD opened file — datasets:")
    names = list(sd.datasets().keys())


/pesq/dados/monan/users/madeleine.gacita/global_data/CER_SYN1deg-MHour/Terra-Aqua-MODIS_Edition4A/CER_SYN1deg-MHour_Terra-Aqua-MODIS_Edition4A_407406.201812.hdf
pyhdf.SD opened file — datasets:


# Plotting radiation var

In [12]:
# Define type aliases
RegionName: TypeAlias = str
LatitudeRange: TypeAlias = tuple[float, float]
LongitudeRange: TypeAlias = tuple[float, float]

# Dictionary of regions with latitude and longitude intervals
regions = {
    "A": {"latitude": (-2, 0), "longitude": (-54, -50)},
    "B": {"latitude": (-3, -1), "longitude": (-61, -58.5)},
    "C": {"latitude": (-4, -2), "longitude": (-67, -64)},
    "D": {"latitude": (-5, -3), "longitude": (-72, -70)},
    "E": {"latitude": (-14, -11), "longitude": (-74, -71.5)},
    "F": {"latitude": (-23, -21), "longitude": (-49, -46)},
}

### Plots general settings

In [13]:
var = "ISR"

flux_var_name = f'{rad_var_dict[var]["era5_name"]}_flux'
fig_path="/pesq/dados/monan/users/madeleine.gacita/figuras_rodada/"


# UTC_hour_plot=15

# Define South America extent
extent = [-85, -30, -35, 15]  # [lon_min, lon_max, lat_min, lat_max]

## Opening MONAN data and extracting {var}

In [14]:
monan_dir="/pesq/dados/monan/users/lianet.hernandez/global_clm_2018-2019/derived_data/"

# monan_diag=f"MONAN_DIAG_G_MOD_GFS_2018111500_{year}{month}{day}{hour}.00.00.x655362L55.nc"
monan_var=rad_var_dict[var]["monan_name"]
monan_file=f'{monan_dir}/{month_abbr}{year}_hourly_{monan_var}_from_accum_MHourly.nc'

monan_path = Path(monan_file)

ds_monan = xr.open_dataset(monan_path, engine="h5netcdf")
# Adjusting sign of OLR / OLRC as negative outgoing
if var=="OLR" or var=="OLRC":
    ds_monan[monan_var] = 0 - ds_monan[monan_var]

print("[INFO] Calculating local time hour for each cell...")
lon_offset_hours = ds_monan["lon"].values / 15.0  # Longitude offset in hours

# For each cell, calculate the local hour for each UTC hour
# Result shape: (hour, nCells)
utc_hours = ds_monan["hour"].values  # 0-23
local_hour = (utc_hours[:, None] + lon_offset_hours[None, :]) % 24
# Cast to integer hours
local_hour = local_hour.astype(int)
# Assign as coordinate to dataset
ds_monan = ds_monan.assign_coords(local_hour=(("hour", "nCells"), local_hour))

monan_lons=ds_monan['lon']
monan_lats=ds_monan['lat']

# Create triangulation
tri = Triangulation(monan_lons, monan_lats)


[INFO] Calculating local time hour for each cell...


### Plotting diurnal cycle for lat, lon interval

### Selecting region data

In [15]:
def select_cells_in_region(ds, region_name):
    lat_min, lat_max = regions[region_name]["latitude"]
    lon_min, lon_max = regions[region_name]["longitude"]
    
    # Renames
    if "longitude" in ds.dims and "latitude" in ds.dims:
        ds = ds.rename({"longitude": "lon"})
        ds = ds.rename({"latitude": "lat"})    
        renamed=True
    else:
        renamed=False
    
    # Create a mask for cells within Region
    mask = ((ds.lat >= lat_min) & (ds.lat <= lat_max) &
            (ds.lon >= lon_min) & (ds.lon <= lon_max))
    # Apply mask to select only cells in Region
    ds_region = ds.where(mask, drop=True)
    if renamed:
        ds_region = ds_region.rename({"lon": "longitude"})
        ds_region = ds_region.rename({"lat": "latitude"})           
    return ds_region


In [16]:
from typing_extensions import TypeAlias

xr.set_options(use_flox=False)

## Extracting CERES {var} data from opened file

In [17]:
ceres_var_name = rad_var_dict[var]['ceres_name']
print(f"Reading dataset: {ceres_var_name}")
sds = sd.select(ceres_var_name)
CER_data = sds.get()              
print(f"{sds.info()}") # shows name, rank, dims, type
dims = sds.info()[2]

#     # Apply scale/offset if present in attributes
attrs = sds.attributes()

scale = attrs.get('scale_factor') or attrs.get('scale') or attrs.get('SCALE')
offset = attrs.get('add_offset') or attrs.get('offset') or attrs.get('OFFSET')
fillvalue  = attrs.get('fill_value') or attrs.get('_FillValue')
datamin,datamax = attrs.get('valid_range')
unit = attrs.get('units')

if scale is not None:
    CER_data = CER_data * float(scale)
if offset is not None:
    CER_data = CER_data + float(offset)
CER_data_masked = np.where(CER_data < datamax, CER_data, np.nan) 

if unit == "cm":
    # converting to kg/m2
    CER_data_masked = CER_data_masked * 10
elif unit == "g m-2":
    # converting to kg/m2
    CER_data_masked = CER_data_masked / 1000

# Find longitude and latitude coordinates for georeferenced plotting
lon_coords = None
lat_coords = None
lon_names = [n for n in names if 'lon' in n.lower() or 'longitude' in n.lower()]
for lname in lon_names:
    try:
        lds = sd.select(lname)
        try:
            lvals = lds.get()
        except Exception:
            lvals = lds[:]
        lon_coords = np.array(lvals)
        try:
            lds.end()
        except Exception:
            pass
        break
    except Exception:
        continue
lat_names = [n for n in names if 'lat' in n.lower() or 'latitude' in n.lower()]
for lname in lat_names:
    try:
        lds = sd.select(lname)
        try:
            lvals = lds.get()
        except Exception:
            lvals = lds[:]
        lat_coords = np.array(lvals)
        try:
            lds.end()
        except Exception:
            pass
        break
    except Exception:
        continue

#     Getting GMT time
hour_data = sd.select('gmt_hr_index').get()

if (len(dims)>3 and 5 in dims):      
#     We have a cloud layer dim
    cld_index = sd.select('cloud_layer').get()


# # Creating Xarray
if (len(dims)>3 and 5 in dims):      
#     Have a cloud layer dim
    if "CER_data_xa" in locals() and isinstance(locals()["CER_data_xa"], xr.Dataset):
        print(f'Adding {ceres_var_name} to dataArray')
        CER_data_xa[ceres_var_name] = (("cloud_layer","gmt_hr_index", "lat", "lon"), CER_data_masked)
    else:
        print('Creating DataSet')
        CER_data_xa = xr.Dataset(
            {
                ceres_var_name: (("cloud_layer", "gmt_hr_index", "lat", "lon"), CER_data_masked)
            },
            coords={
                "cloud_layer": cld_index,
                "gmt_hr_index": hour_data,
                "lat": lat_coords,
                "lon": lon_coords,
            }
        )
else:
#     Have a cloud layer dim
    if "CER_data_xa" in locals() and isinstance(locals()["CER_data_xa"], xr.Dataset):
        print(f'Adding {ceres_var_name} to dataArray')
        CER_data_xa[ceres_var_name] = (("gmt_hr_index", "lat", "lon"), CER_data_masked)
    else:
        print('Creating DataSet')
        CER_data_xa = xr.Dataset(
            {
                ceres_var_name: (( "gmt_hr_index", "lat", "lon"), CER_data_masked)
            },
            coords={
                "gmt_hr_index": hour_data,
                "lat": lat_coords,
                "lon": lon_coords,
            }
        )

# Adjusting sign of OLR / OLRC as negative outgoing
if var=="OLR" or var=="OLRC":
    CER_data_xa[ceres_var_name] = 0 - CER_data_xa[ceres_var_name]

utc_hours = CER_data_xa['gmt_hr_index'].values   # shape (24,)
lon = CER_data_xa['lon'].values                  # shape (360,)

# Longitude offset in hours
lon_offset_hours = lon / 15.0

# Broadcast to (gmt_hr_index, lon)
local_hour = (utc_hours[:, None] + lon_offset_hours[None, :]) % 24
local_hour = local_hour.astype(int)

# Assign as coordinate
CER_data_xa = CER_data_xa.assign_coords(local_hour=(("gmt_hr_index", "lon"), local_hour))

Reading dataset: init_all_sfc_sw_dn
('init_all_sfc_sw_dn', 3, [24, 180, 360], 5, 5)
Creating DataSet


## Opening ERA5 {var} data

In [18]:
era5_file_arg = base_dir+f"era5/single_levels/{year}/{month}/{rad_var_dict[var]['era5_longname']}_MHour.nc"
path = Path(era5_file_arg)

ds_era5 = xr.open_dataset(path, engine="netcdf4")
ds_era5 = apply_lon_lat_conventions(ds_era5)

ds_era5[flux_var_name] = ds_era5[rad_var_dict[var]["era5_name"]]/3600
ds_era5[flux_var_name].attrs = {'units': rad_var_dict[var]["unit"]}
ds_era5[flux_var_name].attrs['description'] = f'Instantaneous flux calculated from accumulated {rad_var_dict[var]["era5_name"].upper()}'

print(ds_era5)
utc_hours_era5 = ds_era5['valid_time'].dt.hour.values   # shape (24,)
lon_era5 = ds_era5['longitude'].values                  # shape (360,)

# Longitude offset in hours
lon_offset_hours_era5 = lon_era5 / 15.0

# Broadcast to (gmt_hr_index, lon)
local_hour_era5 = (utc_hours_era5[:, None] + lon_offset_hours_era5[None, :]) % 24
local_hour_era5 = local_hour_era5.astype(int)

# Assign as coordinate
ds_era5 = ds_era5.assign_coords(local_hour=(("valid_time", "longitude"), local_hour_era5))


<xarray.Dataset> Size: 199MB
Dimensions:     (valid_time: 24, latitude: 721, longitude: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 192B 2018-12-01T01:00:00 ... 2018...
  * latitude    (latitude) float64 6kB -90.0 -89.75 -89.5 ... 89.5 89.75 90.0
    number      int64 8B ...
    expver      (valid_time) <U4 384B ...
  * longitude   (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
Data variables:
    ssrd        (valid_time, latitude, longitude) float32 100MB 1.529e+06 ......
    ssrd_flux   (valid_time, latitude, longitude) float32 100MB 424.8 ... 0.0
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-28T17:22 GRIB to CDM+CF via cfgrib-0.9.1...


## Plotting diurnal cycle for region

### Extract ERA5 Mhourly diurnal cycle 

### Create the plot with all datasets

In [19]:
region_name = "A"

In [20]:
ds_region_monan=select_cells_in_region(ds_monan, region_name)

# Average over cells to get UTC diurnal cycle
avg_region_monan_utc = ds_region_monan[monan_var].mean(dim="nCells")

# Average over cells grouped by local hour
avg_region_monan_local = ds_region_monan[monan_var].groupby('local_hour').mean(dim="stacked_hour_nCells")

In [21]:
# Get the data variable for region
ds_region_ceres = select_cells_in_region(CER_data_xa, region_name)

# Average over cells to get UTC diurnal cycle
avg_region_ceres_utc = ds_region_ceres[ceres_var_name].mean(dim=["lat","lon"]).groupby('gmt_hr_index').mean()

# Average over cells grouped by local hour
# avg_region_ceres_local = ds_region_ceres[ceres_var_name].groupby('local_hour').mean()
avg_region_ceres_local = ds_region_ceres[ceres_var_name].mean(dim="lat").groupby('local_hour').mean()


In [22]:
# Get the data variable for region
ds_region_era5 = select_cells_in_region(ds_era5, region_name)

# Average over cells to get UTC diurnal cycle
avg_region_era5_utc = ds_region_era5[flux_var_name].mean(dim=["latitude","longitude"]).groupby('valid_time').mean()

# Average over cells grouped by local hour
# avg_region_ceres_local = ds_region_ceres[ceres_var_name].groupby('local_hour').mean()
avg_region_era5_local = ds_region_era5[flux_var_name].mean(dim="latitude").groupby('local_hour').mean()


In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(avg_region_monan_local['local_hour'].values, avg_region_monan_local.values, label="MONAN", linewidth=2)
plt.plot(avg_region_ceres_local['local_hour'].values, avg_region_ceres_local.values, label=f"CERES", linewidth=2)
plt.plot(avg_region_era5_local['local_hour'].values, avg_region_era5_local.values, label=f"ERA5", linewidth=2)
plt.ylabel(f"{rad_var_dict[var]['label']} (${rad_var_dict[var]['unit']})$")
plt.xlabel('Local time')
plt.title(f" Monthly Mean diurnal cycle \n Region {region_name} {year}-{month}")
plt.ylim(rad_var_dict[var]['vmin'], rad_var_dict[var]['vmax'])
plt.grid(True, alpha=0.3)
plt.legend()

# Save the plot
out_png = f"{fig_path}/diurnal_cycle_region_{region_name}_{var}_{year}{month}.png"
plt.savefig(out_png, dpi=150, bbox_inches='tight')
print(f"Saved plot to: {out_png}")
plt.show()
plt.close()

### Closing MONAN and ERA5 files

In [251]:
ds_monan.close()
ds_era5.close()

### Closing CERES file

In [252]:
try:
    sd.end()
except Exception:
    pass